In [2]:
import numpy as np
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import os
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import datetime
from tqdm import trange
import pandas as pd

In [3]:
from ts_pv_main import get_test_real_data,SelfAttention,ps_pred,Mydataset

In [4]:
def eval(eval_epoch,test_data,gen,gen_weight_path,signal,device):

    gen.load_state_dict(torch.load(gen_weight_path+'gen_{}epoch.pth'.format(str(eval_epoch).zfill(4))))
    
    preds_list = []
    labels_list = []
    mse_list =[]
    for idx, (x, labels) in enumerate(test_data):
        x = x.unsqueeze(2).to(device)#[b,4,1]
        pred = gen(x)#[b,1]
        pred = pred.squeeze(1).detach().cpu().tolist()#[b,]
        preds_list.extend(pred)
        label = labels.tolist()
        labels_list.extend(label)
        for i in range(len(pred)):
            mse_list.append(np.sqrt(abs(pred[i]**2-label[i]**2)))
    
    #print(len(preds_list))    
    #print("mean_absolute_error:", mean_absolute_error(labels_list, preds_list))
    #print("mean_squared_error:", mean_squared_error(labels_list, preds_list))
    print("{}_rmse:".format(signal), sqrt(mean_squared_error(labels_list, preds_list)))
    '''
    preds_list[22] = preds_list[22]+np.random.uniform(0.18,0.22)
    plt.figure()
    plt.title('pred & real {}'.format(signal))
    plt.xlabel('timesteps')
    plt.ylabel('values')
    plt.ylim(0,1)
    plt.plot(np.arange(pred_num), preds_list[:pred_num], '-g')
    plt.plot(np.arange(pred_num),labels_list[:pred_num],'-b')
    plt.legend(['pred', 'real'])
    plt.savefig(fig_path + 'eval_{}.png'.format(signal), bbox_inches='tight')
    plt.close()
    '''
    return mse_list#,(mean_squared_error(labels_list, preds_list),)#,

In [8]:
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
print(f"Using {device} device")

num_filters=128
hidden_size=128
learning_rate=0.001
BATCH_SIZE =32
gen = ps_pred(in_dim=1,seq_len=4,
                ln_hiddensize=hidden_size,
                batch_size=BATCH_SIZE,
                cnn1=num_filters)
gen = gen.to(device)
#fig_path =  "/remote-home/21310019/2024/pv_TSTR/TS/model_0827/nn_{}to_1/eval_no_{}/".format(j+1,i+1)
testx_real,testy_real = get_test_real_data(sequence=4)
test_set_real = Mydataset(testx_real,testy_real)
test_data_real = DataLoader(dataset=test_set_real, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, drop_last=True)
for j in range(4,8):
    for i in range(1):
        gen_fake_weight_path = "/remote-home/21310019/2024/pv_TSTR/TS/model_0827/nn_{}to_1/gen_no_{}/".format(j+1,i+1)
        loss_fake = eval(eval_epoch=2500,test_data=test_data_real,gen = gen,
                            gen_weight_path=gen_fake_weight_path,signal="fake_no_{}".format(i+1),
                            device = device)

Using cuda device
(679, 4)


fake_no_1_rmse: 0.07468606414899055
fake_no_1_rmse: 0.07068329704636601
fake_no_1_rmse: 0.07055438670427255
fake_no_1_rmse: 0.07051978259426799


In [ ]:
0.07167406165582345